# Constraint Satisfaction Problems

This notebook serves as a supporting material for Chapter 6 **Constraint Satisfaction Problems**. In this notebook, we demonstrate how the code repository can be used to solve problems that can be formulated as CSPs. We use **factored representation** for the states in a CSP. We will see that by deviating from the notion of atomic states, we can use general purpose inference techniques and search heuristics to solve any CSP. Hence, a CSP solver can be used to solve any problem once it has been formulated as a CSP. Let's begin by loading the `aima-core` jar file.

In [1]:
%classpath add jar ../out/artifacts/aima_core_jar/aima-core.jar

# Defining constraint Satisfaction Problems 